# Ray RLlib Tutorial - Exercise Solutions

© 2019-2021, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademyLogo.png)

This notebook contains the solutions for all the exercises in the RLlib tutorial.

First, we have to setup everything needed from the other notebooks.

In [ ]:
import gym
import numpy as np
import pandas as pd
import json
import sys
import os

## 01 Introduction to Reinforcement Learning

### Exercise 1

Finish implementing the `rollout_policy` function below, which should take an environment *and* a policy. Recall that the *policy* is a function that takes in a *state* and returns an *action*. The main difference is that instead of choosing a **random action**, like we just did (with poor results), the action should be chosen **with the policy** (as a function of the state).

In [ ]:
env = gym.make("CartPole-v1")
print("Created env:", env)

In [ ]:
def rollout_policy(env, policy):
    state = env.reset()
    
    done = False
    cumulative_reward = 0

    # Keep looping as long as the simulation has not finished.
    while not done:
        # Choose a random action (either 0 or 1).
        action = policy(state)
        
        # Take the action in the environment.
        state, reward, done, _ = env.step(action)
        
        # Update the cumulative reward.
        cumulative_reward += reward
        
    # Return the cumulative reward.
    return cumulative_reward

def sample_policy1(state):
    return 0 if state[0] < 0 else 1

def sample_policy2(state):
    return 1 if state[0] < 0 else 0

In [ ]:
reward1 = np.mean([rollout_policy(env, sample_policy1) for _ in range(100)])
reward2 = np.mean([rollout_policy(env, sample_policy2) for _ in range(100)])

print('The first sample policy got an average reward of {}.'.format(reward1))
print('The second sample policy got an average reward of {}.'.format(reward2))

assert 5 < reward1 < 15, ('Make sure that rollout_policy computes the action '
                          'by applying the policy to the state.')
assert 25 < reward2 < 35, ('Make sure that rollout_policy computes the action '
                           'by applying the policy to the state.')

### Exercise 2

The current network and training configuration are too large and heavy-duty for a simple problem like `CartPole`. Modify the configuration to use a smaller network and to speed up the optimization of the surrogate objective. (Fewer SGD iterations and a larger batch size should help.)

In [ ]:
import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
from ray.tune.logger import pretty_print

In [ ]:
ray.init(ignore_reinit_error=True, log_to_driver=False)

Here's one possible set. It takes longer for the max reward to reach 200, so I increased the number of episodes `N` to 10.

In [ ]:
config = DEFAULT_CONFIG.copy()
config["num_workers"] = 3
config["num_sgd_iter"] = 10                       # was 30
config["sgd_minibatch_size"] = 256                # was 128
config["model"]["fcnet_hiddens"] = [20, 20]       # was [100, 100]
config["num_cpus_per_worker"] = 0

In [ ]:
agent = PPOTrainer(config, "CartPole-v1")

In [ ]:
N = 20                # was 10
results = []
episode_data = []
episode_json = []

for n in range(N):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'],  
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    
    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}')

In [ ]:
df = pd.DataFrame(data=episode_data)
df

In [ ]:
df.plot(x="n", y=["episode_reward_mean", "episode_reward_min", "episode_reward_max"], secondary_y=True)

Compare this graph with the graph in the lesson, where we used more computing resources:

![](../../images/rllib/Cart-Pole-Episode-Rewards.png)

Note that we only used 5 episodes before. If you compare the graphs at n=4, you see that this execise solution is training more slowly, but it after N=10, the mean reward grows quickly.

Try it again with slightly larger and/or small neural network layers.

In [ ]:
ray.shutdown()